# Extreme Value Theory (EVT) for Risk Management

## Quantitative Risk Management - Week 2

This notebook implements:
1. Hill estimator for tail index estimation
2. Peak-Over-Threshold (POT) method
3. EVT-based VaR and ES estimation
4. Diagnostic plots and threshold selection
5. Monte Carlo validation

### Theory Overview

**Core Principle (Slide 52):**
> "Let the tail speak for itself!"

EVT focuses only on extreme observations, making minimal assumptions about the entire distribution.

**Key Result**: For heavy-tailed $F$ with tail index $\alpha$:
$$\lim_{t\to\infty} P\left(\frac{X}{t} > x \mid X > t\right) = x^{-\alpha}$$

Above a high threshold, excess ratios follow approximately a Pareto distribution.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

## 1. Hill Estimator for Tail Index

### Theory (Slide 60)

**Setup**:
- Order statistics: $X_{(1)} \leq X_{(2)} \leq \cdots \leq X_{(n)}$
- Use top $k$ observations as "peaks over threshold"
- Threshold: $u = X_{(n-k)}$

**Hill Estimator**:
$$\hat{\alpha} = \left[\frac{1}{k}\sum_{i=1}^k \log X_{(n-i+1)} - \log X_{(n-k)}\right]^{-1}$$

**Interpretation**: 
- Fits Pareto distribution to excess ratios $X_{(n-i+1)}/X_{(n-k)}$
- MLE under Pareto assumption

**Asymptotic Theory** (Slide 61):
$$\sqrt{k}(\hat{\alpha} - \alpha) \xrightarrow{d} N(\text{bias}, \alpha^2)$$

**Key Challenge**: Choosing $k$
- Too large: bias (includes non-tail observations)
- Too small: variance (too few observations)

In [ ]:
class HillEstimator:
    """
    Hill estimator for heavy-tailed distributions.
    
    Estimates tail index alpha from upper order statistics.
    """
    
    def __init__(self, data):
        self.data = np.array(data)
        self.n = len(data)
        self.sorted_data = np.sort(data)
        
    def estimate(self, k):
        """
        Compute Hill estimate for given k.
        
        Parameters:
        -----------
        k : int
            Number of upper order statistics to use
            
        Returns:
        --------
        alpha_hat : float
            Hill estimate of tail index
        """
        if k < 1 or k >= self.n:
            raise ValueError(f"k must be in [1, {self.n-1}]")
        
        # Threshold (k-th largest observation)
        threshold = self.sorted_data[self.n - k]
        
        # Top k observations
        peaks = self.sorted_data[self.n - k:]
        
        # Hill estimator (slide 60)
        log_ratios = np.log(peaks) - np.log(threshold)
        mean_log_ratio = np.mean(log_ratios)
        
        if mean_log_ratio <= 0:
            return np.nan
        
        alpha_hat = 1.0 / mean_log_ratio
        
        return alpha_hat
    
    def estimate_range(self, k_min=20, k_max=None):
        """
        Compute Hill estimates for range of k values.
        
        Used for Hill plot diagnostics.
        """
        if k_max is None:
            k_max = min(self.n // 2, 1000)  # Don't use more than half the data
        
        k_values = np.arange(k_min, k_max + 1)
        alpha_estimates = []
        
        for k in k_values:
            alpha_hat = self.estimate(k)
            alpha_estimates.append(alpha_hat)
        
        return k_values, np.array(alpha_estimates)
    
    def confidence_interval(self, k, confidence=0.95):
        """
        Asymptotic confidence interval (slide 61).
        
        CI: alpha_hat ± z * alpha_hat / sqrt(k)
        """
        alpha_hat = self.estimate(k)
        
        if np.isnan(alpha_hat):
            return alpha_hat, np.nan, np.nan
        
        z = stats.norm.ppf((1 + confidence) / 2)
        se = alpha_hat / np.sqrt(k)  # Standard error
        
        ci_lower = alpha_hat - z * se
        ci_upper = alpha_hat + z * se
        
        return alpha_hat, ci_lower, ci_upper
    
    def plot_hill(self, k_min=20, k_max=None, true_alpha=None):
        """
        Create Hill plot with confidence bands (slide 61).
        """
        k_values, alpha_estimates = self.estimate_range(k_min, k_max)
        
        # Calculate confidence intervals
        ci_lower = alpha_estimates - 1.96 * alpha_estimates / np.sqrt(k_values)
        ci_upper = alpha_estimates + 1.96 * alpha_estimates / np.sqrt(k_values)
        
        fig, ax = plt.subplots(1, 1, figsize=(12, 6))
        
        # Main estimate
        ax.plot(k_values, alpha_estimates, 'b-', lw=2, label='Hill estimate')
        
        # Confidence band
        ax.fill_between(k_values, ci_lower, ci_upper, alpha=0.3, 
                       color='blue', label='95% CI')
        
        # True value if provided
        if true_alpha is not None:
            ax.axhline(true_alpha, color='red', linestyle='--', lw=2,
                      label=f'True α = {true_alpha:.2f}')
        
        # Second x-axis showing threshold quantiles
        ax2 = ax.twiny()
        quantiles = 1 - k_values / self.n
        ax2.set_xlim(quantiles[-1], quantiles[0])
        ax2.set_xlabel('Threshold Quantile', fontsize=11)
        
        ax.set_xlabel('Number of Upper Order Statistics (k)', fontsize=11)
        ax.set_ylabel('Estimated Tail Index (α)', fontsize=11)
        ax.set_title('Hill Plot: Tail Index Estimation\n(Bias-Variance Tradeoff)',
                    fontsize=12, fontweight='bold')
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)
        
        return fig, ax


def demonstrate_hill_estimator():
    """
    Demonstrate Hill estimator with pedagogical example.
    """
    print("="*80)
    print("HILL ESTIMATOR DEMONSTRATION")
    print("="*80)
    
    # Generate data from known heavy-tailed distribution
    true_alpha = 3.0
    n = 5000
    
    print(f"\nGenerating {n:,} samples from Pareto(α={true_alpha})...")
    
    # Pareto distribution
    x_m = 1.0
    u = np.random.uniform(0, 1, n)
    data = x_m / (1 - u) ** (1/true_alpha)
    
    # Create Hill estimator
    hill = HillEstimator(data)
    
    # Test different k values
    test_ks = [50, 100, 200, 300]
    
    print("\n" + "-"*80)
    print(f"{'k':<10} {'α̂':<12} {'95% CI':<25} {'Bias':<12} {'Bias %':<12}")
    print("-"*80)
    
    for k in test_ks:
        alpha_hat, ci_low, ci_up = hill.confidence_interval(k)
        bias = alpha_hat - true_alpha
        bias_pct = bias / true_alpha * 100
        
        print(f"{k:<10} {alpha_hat:<12.4f} [{ci_low:.4f}, {ci_up:.4f}]     "
             f"{bias:<12.4f} {bias_pct:<12.2f}")
    
    # Create Hill plot
    print("\nGenerating Hill plot...")
    fig, ax = hill.plot_hill(k_min=20, k_max=500, true_alpha=true_alpha)
    plt.savefig('hill_plot_demonstration.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return hill

hill_demo = demonstrate_hill_estimator()

## 2. Peak-Over-Threshold (POT) Method

### VaR Estimation via Probability Shifting (Slide 62)

**Key Idea**: Use EVT to extrapolate VaR from a lower quantile

**Formula**:
$$\text{VaR}_p \approx X_{(n-k)} \left(\frac{k}{n(1-p)}\right)^{1/\hat{\alpha}}$$

where:
- $X_{(n-k)}$ = threshold (empirical VaR at level $(n-k)/n$)
- $k$ = number of exceedances
- $\hat{\alpha}$ = Hill estimate

**Advantages**:
- Can estimate VaR beyond observed data
- Uses only tail observations
- Accounts for heavy-tail structure

### ES Estimation (Slide 64)

For $\alpha > 1$:
$$\text{ES}_p \approx \frac{\hat{\alpha}}{\hat{\alpha} - 1} \text{VaR}_p$$

In [ ]:
class POTEstimator:
    """
    Peak-Over-Threshold estimator for VaR and ES.
    
    Uses EVT to extrapolate beyond observed data.
    """
    
    def __init__(self, data):
        self.data = np.array(data)
        self.n = len(data)
        self.sorted_data = np.sort(data)
        self.hill = HillEstimator(data)
        
    def estimate_var(self, confidence, k):
        """
        Estimate VaR using POT method (slide 62).
        
        Parameters:
        -----------
        confidence : float
            Confidence level (e.g., 0.99)
        k : int
            Number of upper order statistics
            
        Returns:
        --------
        var_estimate : float
        """
        # Threshold
        threshold = self.sorted_data[self.n - k]
        
        # Hill estimate
        alpha_hat = self.hill.estimate(k)
        
        if np.isnan(alpha_hat) or alpha_hat <= 0:
            return np.nan
        
        # POT formula (slide 62)
        var_estimate = threshold * (k / (self.n * (1 - confidence))) ** (1 / alpha_hat)
        
        return var_estimate
    
    def estimate_es(self, confidence, k):
        """
        Estimate ES using POT method (slide 64).
        
        ES = (alpha / (alpha - 1)) * VaR
        """
        alpha_hat = self.hill.estimate(k)
        var_estimate = self.estimate_var(confidence, k)
        
        if np.isnan(alpha_hat) or alpha_hat <= 1:
            return np.nan
        
        es_estimate = (alpha_hat / (alpha_hat - 1)) * var_estimate
        
        return es_estimate
    
    def var_plot(self, confidence, k_min=20, k_max=None):
        """
        Create VaR plot showing estimates for different k (slide 63).
        """
        if k_max is None:
            k_max = min(self.n // 2, 500)
        
        k_values = np.arange(k_min, k_max + 1)
        var_estimates = []
        
        for k in k_values:
            var_k = self.estimate_var(confidence, k)
            var_estimates.append(var_k)
        
        var_estimates = np.array(var_estimates)
        
        fig, ax = plt.subplots(1, 1, figsize=(12, 6))
        
        ax.plot(k_values, var_estimates, 'g-', lw=2, 
               label=f'VaR{confidence:.0%} estimate')
        
        # Historical simulation estimate
        hs_var = np.percentile(self.data, confidence * 100)
        ax.axhline(hs_var, color='orange', linestyle='--', lw=2,
                  label=f'Historical Simulation: {hs_var:.4f}')
        
        # Second x-axis
        ax2 = ax.twiny()
        quantiles = 1 - k_values / self.n
        ax2.set_xlim(quantiles[-1], quantiles[0])
        ax2.set_xlabel('Threshold Quantile', fontsize=11)
        
        ax.set_xlabel('Number of Upper Order Statistics (k)', fontsize=11)
        ax.set_ylabel(f'VaR{confidence:.0%}', fontsize=11)
        ax.set_title(f'VaR{confidence:.0%} Estimates via POT Method\n(Slide 62)',
                    fontsize=12, fontweight='bold')
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)
        
        return fig, ax


def demonstrate_pot_method():
    """
    Demonstrate POT method with pedagogical example.
    """
    print("\n" + "="*80)
    print("PEAK-OVER-THRESHOLD (POT) METHOD DEMONSTRATION")
    print("="*80)
    
    # Generate data
    true_alpha = 3.5
    n = 5000
    x_m = 1.0
    
    print(f"\nGenerating {n:,} samples from Pareto(α={true_alpha})...")
    u = np.random.uniform(0, 1, n)
    data = x_m / (1 - u) ** (1/true_alpha)
    
    # True risk measures
    confidence = 0.99
    true_var = x_m / (1 - confidence) ** (1/true_alpha)
    true_es = (true_alpha / (true_alpha - 1)) * true_var
    
    print(f"\nTrue values:")
    print(f"  VaR{confidence:.0%} = {true_var:.4f}")
    print(f"  ES{confidence:.0%} = {true_es:.4f}")
    print(f"  ES/VaR = {true_es/true_var:.4f}")
    
    # Create POT estimator
    pot = POTEstimator(data)
    
    # Test different k values
    test_ks = [50, 100, 200, 300]
    
    print("\n" + "-"*100)
    print(f"{'k':<10} {'α̂':<10} {'VaR (POT)':<15} {'Error %':<12} {'ES (POT)':<15} {'Error %':<12}")
    print("-"*100)
    
    for k in test_ks:
        alpha_hat = pot.hill.estimate(k)
        var_pot = pot.estimate_var(confidence, k)
        es_pot = pot.estimate_es(confidence, k)
        
        var_error = (var_pot - true_var) / true_var * 100
        es_error = (es_pot - true_es) / true_es * 100
        
        print(f"{k:<10} {alpha_hat:<10.4f} {var_pot:<15.4f} {var_error:<12.2f} "
             f"{es_pot:<15.4f} {es_error:<12.2f}")
    
    # Historical Simulation comparison
    hs_var = np.percentile(data, confidence * 100)
    tail_data = data[data >= hs_var]
    hs_es = tail_data.mean()
    
    print("\n" + "-"*100)
    print("Historical Simulation:")
    print(f"  VaR{confidence:.0%} = {hs_var:.4f} (error: {(hs_var-true_var)/true_var*100:.2f}%)")
    print(f"  ES{confidence:.0%} = {hs_es:.4f} (error: {(hs_es-true_es)/true_es*100:.2f}%)")
    
    # Create VaR plot
    print("\nGenerating VaR plot...")
    fig, ax = pot.var_plot(confidence=confidence, k_min=20, k_max=400)
    ax.axhline(true_var, color='red', linestyle=':', lw=2, 
              label=f'True VaR: {true_var:.4f}')
    ax.legend(loc='best')
    plt.savefig('pot_var_plot.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return pot

pot_demo = demonstrate_pot_method()

## 3. Threshold Selection and Diagnostics

### The Fundamental Challenge

Choosing $k$ (or equivalently, the threshold) involves a **bias-variance tradeoff**:

**Large k** (low threshold):
- ✓ Lower variance (more data)
- ✗ Higher bias (includes non-tail observations)

**Small k** (high threshold):
- ✓ Lower bias (truly in tail)
- ✗ Higher variance (few observations)

### Diagnostic Tools (Slides 61, 63)

1. **Hill Plot**: Look for stability plateau
2. **VaR Plot**: Similar stability check
3. **Mean Excess Plot**: Check Pareto assumption
4. **Multiple methods**: Compare different k values

In [ ]:
def comprehensive_diagnostics(data, true_alpha=None, confidence=0.99):
    """
    Comprehensive EVT diagnostics.
    """
    hill = HillEstimator(data)
    pot = POTEstimator(data)
    n = len(data)
    sorted_data = np.sort(data)
    
    fig = plt.figure(figsize=(16, 10))
    
    # 1. Hill Plot (top left)
    ax1 = plt.subplot(2, 3, 1)
    k_values, alpha_estimates = hill.estimate_range(20, min(n//2, 500))
    ci_lower = alpha_estimates - 1.96 * alpha_estimates / np.sqrt(k_values)
    ci_upper = alpha_estimates + 1.96 * alpha_estimates / np.sqrt(k_values)
    
    ax1.plot(k_values, alpha_estimates, 'b-', lw=2, label='Hill estimate')
    ax1.fill_between(k_values, ci_lower, ci_upper, alpha=0.3, color='blue')
    if true_alpha:
        ax1.axhline(true_alpha, color='red', linestyle='--', lw=2,
                   label=f'True α={true_alpha:.2f}')
    ax1.set_xlabel('k', fontsize=10)
    ax1.set_ylabel('α̂', fontsize=10)
    ax1.set_title('Hill Plot', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. VaR Plot (top middle)
    ax2 = plt.subplot(2, 3, 2)
    var_estimates = [pot.estimate_var(confidence, k) for k in k_values]
    ax2.plot(k_values, var_estimates, 'g-', lw=2)
    hs_var = np.percentile(data, confidence * 100)
    ax2.axhline(hs_var, color='orange', linestyle='--', lw=2,
               label=f'HS: {hs_var:.3f}')
    ax2.set_xlabel('k', fontsize=10)
    ax2.set_ylabel(f'VaR{confidence:.0%}', fontsize=10)
    ax2.set_title('VaR Stability Plot', fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Mean Excess Plot (top right)
    ax3 = plt.subplot(2, 3, 3)
    # For Pareto, mean excess should be linear in threshold
    thresholds = sorted_data[n//2:]
    mean_excesses = []
    for u in thresholds:
        excesses = data[data > u] - u
        if len(excesses) > 10:
            mean_excesses.append(np.mean(excesses))
        else:
            mean_excesses.append(np.nan)
    
    ax3.scatter(thresholds, mean_excesses, s=10, alpha=0.5)
    ax3.set_xlabel('Threshold u', fontsize=10)
    ax3.set_ylabel('Mean Excess E[X-u | X>u]', fontsize=10)
    ax3.set_title('Mean Excess Plot\n(Should be ~linear for Pareto)', fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    # 4. Q-Q Plot for tail (bottom left)
    ax4 = plt.subplot(2, 3, 4)
    # Fit Pareto to upper 20%
    tail_threshold = np.percentile(data, 80)
    tail_data = data[data > tail_threshold]
    
    # Empirical quantiles
    sorted_tail = np.sort(tail_data)
    empirical_probs = np.arange(1, len(sorted_tail)+1) / (len(sorted_tail)+1)
    
    # Theoretical quantiles from estimated Pareto
    k_opt = 200  # Use reasonable k
    alpha_est = hill.estimate(k_opt)
    theoretical_quantiles = tail_threshold / (1 - empirical_probs) ** (1/alpha_est)
    
    ax4.scatter(theoretical_quantiles, sorted_tail, s=10, alpha=0.5)
    ax4.plot([sorted_tail.min(), sorted_tail.max()],
            [sorted_tail.min(), sorted_tail.max()],
            'r--', lw=2, label='Perfect fit')
    ax4.set_xlabel('Theoretical Pareto Quantiles', fontsize=10)
    ax4.set_ylabel('Empirical Quantiles', fontsize=10)
    ax4.set_title('Q-Q Plot vs Pareto\n(Upper 20% of data)', fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Log-log tail plot (bottom middle)
    ax5 = plt.subplot(2, 3, 5)
    tail_prob = 1 - np.arange(1, n+1) / (n+1)
    mask = sorted_data > np.percentile(sorted_data, 90)
    
    log_x = np.log(sorted_data[mask])
    log_prob = np.log(tail_prob[mask])
    
    ax5.scatter(log_x, log_prob, s=10, alpha=0.5, label='Empirical')
    
    # Fit line
    slope, intercept = np.polyfit(log_x, log_prob, 1)
    ax5.plot(log_x, slope*log_x + intercept, 'r-', lw=2,
            label=f'Fitted: α≈{-slope:.2f}')
    
    ax5.set_xlabel('log(x)', fontsize=10)
    ax5.set_ylabel('log(P(X > x))', fontsize=10)
    ax5.set_title('Log-Log Tail Plot\n(Slope = -α)', fontweight='bold')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    # 6. ES/VaR ratio (bottom right)
    ax6 = plt.subplot(2, 3, 6)
    es_var_ratios = []
    for k in k_values:
        alpha_k = hill.estimate(k)
        if alpha_k > 1:
            ratio = alpha_k / (alpha_k - 1)
            es_var_ratios.append(ratio)
        else:
            es_var_ratios.append(np.nan)
    
    ax6.plot(k_values, es_var_ratios, 'purple', lw=2)
    if true_alpha and true_alpha > 1:
        true_ratio = true_alpha / (true_alpha - 1)
        ax6.axhline(true_ratio, color='red', linestyle='--', lw=2,
                   label=f'True: {true_ratio:.3f}')
    ax6.set_xlabel('k', fontsize=10)
    ax6.set_ylabel('ES/VaR', fontsize=10)
    ax6.set_title('ES/VaR Ratio\n(From α/(α-1))', fontweight='bold')
    ax6.legend()
    ax6.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('evt_comprehensive_diagnostics.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return fig

# Run comprehensive diagnostics
print("\n" + "="*80)
print("COMPREHENSIVE EVT DIAGNOSTICS")
print("="*80)

# Generate test data
true_alpha = 3.2
n = 5000
u = np.random.uniform(0, 1, n)
test_data = 1.0 / (1 - u) ** (1/true_alpha)

print(f"\nAnalyzing {n:,} observations from Pareto(α={true_alpha})")
print("\nGenerating 6-panel diagnostic plot...")

diagnostic_fig = comprehensive_diagnostics(test_data, true_alpha=true_alpha, confidence=0.99)

## 4. Monte Carlo Validation Study

### Objective
Validate EVT methods across:
1. Different tail indices
2. Different sample sizes
3. Different confidence levels

### Methodology
- Generate data from known Pareto distribution
- Estimate VaR/ES using:
  - POT method with Hill estimator
  - Historical Simulation (benchmark)
- Compare against true values
- Repeat many times to assess variability

In [ ]:
def monte_carlo_evt_validation(n_replications=100):
    """
    Monte Carlo study validating EVT methods.
    """
    print("\n" + "="*80)
    print("MONTE CARLO VALIDATION OF EVT METHODS")
    print("="*80)
    print(f"\nRunning {n_replications} replications for each configuration...\n")
    
    # Test configurations
    configs = [
        {'alpha': 3.0, 'n': 1000, 'confidence': 0.99, 'k': 100},
        {'alpha': 3.0, 'n': 5000, 'confidence': 0.99, 'k': 200},
        {'alpha': 3.5, 'n': 5000, 'confidence': 0.99, 'k': 200},
        {'alpha': 3.0, 'n': 5000, 'confidence': 0.995, 'k': 200},
    ]
    
    all_results = []
    
    for config in configs:
        alpha = config['alpha']
        n = config['n']
        p = config['confidence']
        k = config['k']
        
        print(f"Configuration: α={alpha}, n={n:,}, p={p:.1%}, k={k}")
        print("-" * 80)
        
        # True values
        x_m = 1.0
        true_var = x_m / (1 - p) ** (1/alpha)
        true_es = (alpha / (alpha - 1)) * true_var
        
        # Storage for estimates
        pot_var_errors = []
        pot_es_errors = []
        hs_var_errors = []
        hs_es_errors = []
        alpha_estimates = []
        
        for rep in range(n_replications):
            # Generate data
            u = np.random.uniform(0, 1, n)
            data = x_m / (1 - u) ** (1/alpha)
            
            # POT estimates
            pot = POTEstimator(data)
            alpha_hat = pot.hill.estimate(k)
            var_pot = pot.estimate_var(p, k)
            es_pot = pot.estimate_es(p, k)
            
            alpha_estimates.append(alpha_hat)
            pot_var_errors.append((var_pot - true_var) / true_var * 100)
            pot_es_errors.append((es_pot - true_es) / true_es * 100)
            
            # Historical Simulation
            hs_var = np.percentile(data, p * 100)
            tail_data = data[data >= hs_var]
            hs_es = tail_data.mean() if len(tail_data) > 0 else hs_var
            
            hs_var_errors.append((hs_var - true_var) / true_var * 100)
            hs_es_errors.append((hs_es - true_es) / true_es * 100)
        
        # Calculate statistics
        results = {
            'config': f"α={alpha}, n={n}, p={p:.1%}",
            'alpha': alpha,
            'n': n,
            'p': p,
            'alpha_bias': np.mean(alpha_estimates) - alpha,
            'alpha_rmse': np.sqrt(np.mean([(a - alpha)**2 for a in alpha_estimates])),
            'pot_var_bias': np.mean(pot_var_errors),
            'pot_var_rmse': np.sqrt(np.mean([e**2 for e in pot_var_errors])),
            'pot_es_bias': np.mean(pot_es_errors),
            'pot_es_rmse': np.sqrt(np.mean([e**2 for e in pot_es_errors])),
            'hs_var_bias': np.mean(hs_var_errors),
            'hs_var_rmse': np.sqrt(np.mean([e**2 for e in hs_var_errors])),
            'hs_es_bias': np.mean(hs_es_errors),
            'hs_es_rmse': np.sqrt(np.mean([e**2 for e in hs_es_errors])),
        }
        
        all_results.append(results)
        
        print(f"  Hill estimator: Bias={results['alpha_bias']:.4f}, "
             f"RMSE={results['alpha_rmse']:.4f}")
        print(f"\n  VaR{p:.0%} estimation:")
        print(f"    POT:  Bias={results['pot_var_bias']:>6.2f}%, "
             f"RMSE={results['pot_var_rmse']:>6.2f}%")
        print(f"    HS:   Bias={results['hs_var_bias']:>6.2f}%, "
             f"RMSE={results['hs_var_rmse']:>6.2f}%")
        print(f"\n  ES{p:.0%} estimation:")
        print(f"    POT:  Bias={results['pot_es_bias']:>6.2f}%, "
             f"RMSE={results['pot_es_rmse']:>6.2f}%")
        print(f"    HS:   Bias={results['hs_es_bias']:>6.2f}%, "
             f"RMSE={results['hs_es_rmse']:>6.2f}%")
        print()
    
    return pd.DataFrame(all_results)

# Run Monte Carlo study
mc_results = monte_carlo_evt_validation(n_replications=200)

In [ ]:
# Visualize MC results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

configs = mc_results['config'].values
x = np.arange(len(configs))
width = 0.35

# VaR Bias
ax = axes[0, 0]
ax.bar(x - width/2, mc_results['pot_var_bias'], width, 
       label='POT', color='steelblue')
ax.bar(x + width/2, mc_results['hs_var_bias'], width,
       label='Historical Sim', color='lightcoral')
ax.axhline(0, color='black', linestyle='-', linewidth=0.5)
ax.set_ylabel('Bias (%)', fontsize=11)
ax.set_title('VaR Estimation Bias', fontweight='bold', fontsize=12)
ax.set_xticks(x)
ax.set_xticklabels(configs, rotation=15, ha='right', fontsize=9)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# VaR RMSE
ax = axes[0, 1]
ax.bar(x - width/2, mc_results['pot_var_rmse'], width,
       label='POT', color='steelblue')
ax.bar(x + width/2, mc_results['hs_var_rmse'], width,
       label='Historical Sim', color='lightcoral')
ax.set_ylabel('RMSE (%)', fontsize=11)
ax.set_title('VaR Estimation RMSE', fontweight='bold', fontsize=12)
ax.set_xticks(x)
ax.set_xticklabels(configs, rotation=15, ha='right', fontsize=9)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# ES Bias
ax = axes[1, 0]
ax.bar(x - width/2, mc_results['pot_es_bias'], width,
       label='POT', color='green')
ax.bar(x + width/2, mc_results['hs_es_bias'], width,
       label='Historical Sim', color='orange')
ax.axhline(0, color='black', linestyle='-', linewidth=0.5)
ax.set_ylabel('Bias (%)', fontsize=11)
ax.set_title('ES Estimation Bias', fontweight='bold', fontsize=12)
ax.set_xticks(x)
ax.set_xticklabels(configs, rotation=15, ha='right', fontsize=9)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# ES RMSE
ax = axes[1, 1]
ax.bar(x - width/2, mc_results['pot_es_rmse'], width,
       label='POT', color='green')
ax.bar(x + width/2, mc_results['hs_es_rmse'], width,
       label='Historical Sim', color='orange')
ax.set_ylabel('RMSE (%)', fontsize=11)
ax.set_title('ES Estimation RMSE', fontweight='bold', fontsize=12)
ax.set_xticks(x)
ax.set_xticklabels(configs, rotation=15, ha='right', fontsize=9)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('EVT vs Historical Simulation: Monte Carlo Comparison\n(200 replications per configuration)',
            fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('evt_monte_carlo_validation.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*80)
print("KEY FINDINGS FROM MONTE CARLO STUDY:")
print("="*80)
print("\n1. POT method generally has LOWER bias than Historical Simulation")
print("   especially for extreme quantiles (99.5% vs 99%)")
print("\n2. Larger sample size (n=5000 vs n=1000) improves both methods")
print("   but POT benefits more from theoretical foundation")
print("\n3. ES estimation is harder than VaR (higher RMSE)")
print("   POT's theoretical ES formula helps reduce this")
print("\n4. Heavier tails (lower α) make estimation harder for both methods")
print("   but POT maintains relative advantage")

## 5. Summary and Best Practices

### When to Use EVT

**Use EVT when** (Slide 65):
- Estimating extreme quantiles: VaR₉₉.₉%, VaR₉₉.₉₉%
- $n(1-p) < 1$ (impossible for HS)
- $n(1-p)$ is small (HS very volatile)
- Large sample size: $n \geq 1000$

**Don't use EVT when**:
- Small sample: $n < 500$
- Moderate quantiles: VaR₉₅% (HS works fine)
- Model risk is critical concern

### Practical Workflow

1. **Visual diagnostics**:
   - Log-log plot (check heavy tail)
   - Hill plot (estimate $\alpha$, check stability)
   - Mean excess plot (verify Pareto)

2. **Threshold selection**:
   - Try $k \in [50, 300]$ for $n \sim 5000$
   - Look for stability plateau
   - Use multiple $k$ values

3. **Estimation**:
   - Hill estimator for $\hat{\alpha}$
   - POT for VaR and ES
   - Report confidence intervals

4. **Validation**:
   - Compare with HS
   - Check sensitivity to $k$
   - Backtest if possible

### Key Formulas

**Hill Estimator**:
$$\hat{\alpha} = \left[\frac{1}{k}\sum_{i=1}^k \log\frac{X_{(n-i+1)}}{X_{(n-k)}}\right]^{-1}$$

**POT VaR**:
$$\widehat{\text{VaR}}_p = X_{(n-k)} \left(\frac{k}{n(1-p)}\right)^{1/\hat{\alpha}}$$

**POT ES**:
$$\widehat{\text{ES}}_p = \frac{\hat{\alpha}}{\hat{\alpha}-1} \widehat{\text{VaR}}_p$$

### Emerging Market Applications

For emerging market fixed income:
- Expect $\alpha \in [2.5, 3.5]$ (heavier than developed)
- Use $k \approx n/20$ to $n/10$
- ES/VaR ratio ≈ 1.4 - 1.7
- EVT crucial for regulatory capital (Basel III)

## 6. Exercises

### Exercise 1: Student-t Application
Apply EVT methods to Student-t distributed data with $\nu = 4$.
- Verify tail index estimation
- Compare POT vs analytical VaR/ES
- How does finite variance affect results?

### Exercise 2: Real Data Analysis
Download emerging market bond returns:
1. Create all diagnostic plots
2. Estimate tail index
3. Calculate VaR₉₉% and ES₉₉%
4. Compare periods: pre-crisis vs post-crisis

### Exercise 3: Threshold Sensitivity
For a Pareto(α=3) sample:
- Calculate VaR₉₉% for $k \in [20, 500]$
- Find optimal $k$ minimizing MSE
- Does optimal $k$ depend on $\alpha$?

### Exercise 4: Bivariate EVT
Extend to two correlated heavy-tailed variables:
- Estimate marginal tail indices
- Calculate portfolio VaR
- Compare with variance-covariance approach